In [1]:

import time
import datetime
import requests, json

import numpy as np

import pandas as pd
from datetime import datetime, timedelta
from oceanoobsbrasil.db import GetData
from dotenv import load_dotenv

In [2]:
from oceanoobsbrasil.db import GetData
%load_ext autoreload
%autoreload 2

In [8]:
datetime.utcnow() - timedelta(days=1) < datetime(2022,7,6,9,0,0)

False

In [11]:
start_date = datetime(2022,7,6,9,0,0).strftime('%Y-%m-%d %H:00:00')
start_date

'2022-07-06 09:00:00'

In [3]:
bd = GetData()

In [4]:
bd.get(table='stations', institution=['=', 'gloss'], data_type=['=', 'tide'])

,id,name,lat,lon,data_type,institution,url,flag
0,1157,Fernando de Noronha,-3.833902,-32.40173,tide,gloss,1869,None


In [12]:
start_date=(datetime.utcnow() - timedelta(days=1)).strftime('%Y-%m-%d 00:00:00')
start_date

'2022-06-05 00:00:00'

In [3]:
from oceanoobsbrasil.tides.gloss_tide import GlossTide

In [4]:
g = GlossTide()
g.get()

            date_time         water_level       water_level_2 station_id
0 2022-06-07 09:00:00                 1.7                1.82       1157
1 2022-06-07 09:15:00                1.74                1.86       1157
2 2022-06-07 09:30:00                1.84  1.8900000000000001       1157
3 2022-06-07 09:45:00  1.8800000000000001                1.94       1157
4 2022-06-07 10:00:00                1.97                2.06       1157


In [23]:
station = g.stations.iloc[0]

In [32]:
query = {'startTime': g.start_date,
         'endTime': g.end_date,
         'stationId': str(station.url),
         'sensorName': 'RLS'                    
         }
headers = {'clientId': os.getenv('GLOSS_CLIENT'),
          'api-key': os.getenv('GLOSS_APIKEY')}

In [34]:
response = requests.get('http://www.hydrometcloud.com:8080/Data/rest/api/sensordata', params=query, headers=headers)
response.json()['sensorData']

<Response [200]>

In [39]:
data = response.json()['sensorData']
g.data = pd.DataFrame(data)
g.data

,sampleTime,value
0,2022-06-05 00:00:00,1.27
1,2022-06-05 00:15:00,1.33
2,2022-06-05 00:30:00,1.4000000000000001
3,2022-06-05 00:45:00,1.47
4,2022-06-05 01:00:00,1.51
...,...,...
157,2022-06-06 18:00:00,1.31
158,2022-06-06 18:15:00,1.33
159,2022-06-06 18:30:00,1.29
160,2022-06-06 18:45:00,1.29


In [41]:
query['sensorName'] = 'SE200'
response = requests.get('http://www.hydrometcloud.com:8080/Data/rest/api/sensordata', params=query, headers=headers)
data = response.json()['sensorData']
g.data2 = pd.DataFrame(data)


In [46]:
g.data.set_index('sampleTime', inplace=True)
g.data2.set_index('sampleTime', inplace=True)

In [58]:
g.result = pd.concat([g.data,  g.data2], axis=1, join='inner').reset_index()

In [59]:
g.result.columns = ['date_time', 'water_level', 'water_level_2']

In [60]:
g.result.date_time = pd.to_datetime(g.result.date_time) + pd.DateOffset(hours=3)
g.result

,date_time,water_level,water_level_2
0,2022-06-05 03:00:00,1.27,1.44
1,2022-06-05 03:15:00,1.33,1.3900000000000001
2,2022-06-05 03:30:00,1.4000000000000001,1.5
3,2022-06-05 03:45:00,1.47,1.55
4,2022-06-05 04:00:00,1.51,1.61
...,...,...,...
156,2022-06-06 21:00:00,1.31,1.3
157,2022-06-06 21:15:00,1.33,1.3
158,2022-06-06 21:30:00,1.29,1.28
159,2022-06-06 21:45:00,1.29,1.36


In [57]:
g.result.date_time + pd.DateOffset(hours=3)

0     2022-06-05 03:00:00
1     2022-06-05 03:15:00
2     2022-06-05 03:30:00
3     2022-06-05 03:45:00
4     2022-06-05 04:00:00
              ...        
156   2022-06-06 21:00:00
157   2022-06-06 21:15:00
158   2022-06-06 21:30:00
159   2022-06-06 21:45:00
160   2022-06-06 22:00:00
Name: date_time, Length: 161, dtype: datetime64[ns]

In [3]:
import time
import datetime
import operator
import urllib.request, json

import numpy as np
from oceanoobsbrasil.buoys.simcosta import Simcosta
from oceanoobsbrasil.tides.simcosta import SimcostaTide

In [32]:
s.result

,water_level,wspd,wdir,atmp,rh,pres,date_time,station_id
0,1040.94,0,0,28.9,76,1010.3,2022-04-27 21:41:55+00:00,876
1,1030.74,0,0,28.9,77,1010.4,2022-04-27 21:55:05+00:00,876
2,1017.98,0,0,28.8,77,1010.6,2022-04-27 22:10:06+00:00,876
3,1007.92,0,0,28.9,78,1010.7,2022-04-27 22:25:04+00:00,876
4,1001.21,0,0,28.0,81,1010.9,2022-04-27 22:41:16+00:00,876
...,...,...,...,...,...,...,...,...
334,1077.51,0,0,28.3,79,1012.1,2022-05-01 23:25:01+00:00,876
335,1061.74,0,0,28.5,79,1012.3,2022-05-01 23:40:01+00:00,876
336,1035.95,0,0,29.2,77,1012.6,2022-05-02 00:10:00+00:00,876
337,1023.66,0,0,29.1,78,1012.6,2022-05-02 00:26:11+00:00,876


In [8]:
from datetime import datetime
from pytides.tide import Tide
import numpy as np
import matplotlib.pyplot as plt

#For a quicker decomposition, we'll only use hourly readings rather than 6-minutely readings.
heights = df.water_level
t = df.water_level

##Prepare a list of datetimes, each 6 minutes apart, for a week.
prediction_t0 = datetime(2013,1,1)
hours = 0.1*np.arange(7 * 24 * 10)
times = Tide._times(prediction_t0, hours)

##Fit the tidal data to the harmonic model using Pytides
my_tide = Tide.decompose(heights, t)
##Predict the tides using the Pytides model.
my_prediction = my_tide.at(times)

##Plot the results
plt.plot(hours, my_prediction, label="Pytides")
plt.legend()
plt.title('Comparison of Pytides and NOAA predictions for Station: ' + str(station_id))
plt.xlabel('Hours since ' + str(prediction_t0) + '(GMT)')
plt.ylabel('Metres')
plt.show()

ModuleNotFoundError: No module named 'tide'

In [4]:
s = SimcostaTide()

In [40]:
s.stations

,id,name,lat,lon,data_type,institution,url,flag
0,1156,DHN - RJ,-22.884722,-43.13222,tide,simcosta,304,None
1,873,Rio Grande 1 – RS,-32.188100,-52.08190,tide,simcosta,301,True
2,874,São José do Ribamar – MA,-8.393100,-34.96000,tide,simcosta,302,True
3,875,Tramandaí – RS,-30.005000,-50.12890,tide,simcosta,305,True
4,877,Imbituba – SC,-28.231100,-48.65000,tide,simcosta,308,True
5,878,Ilhabela – SP,-23.773600,-45.35470,tide,simcosta,309,True
6,879,Suape – PE,-8.393100,-34.96000,tide,simcosta,311,True
7,876,Pecém – CE,-3.531100,-38.79310,tide,simcosta,307,True


In [5]:
s.get()

     water_level  wspd  wdir  atmp    rh   pres                 date_time  \
0        1199.01     0     0  None  None  500.5 2022-06-29 15:00:33+00:00   
1        1205.39     0     0  None  None  500.5 2022-06-29 15:14:05+00:00   
2        1214.28     0     0  None  None  500.5 2022-06-29 15:29:03+00:00   
3        1218.92     0     0  None  None  500.5 2022-06-29 15:45:25+00:00   
4        1226.79     0     0  None  None  500.5 2022-06-29 15:59:03+00:00   
..           ...   ...   ...   ...   ...    ...                       ...   
316      1195.11     0     0  None  None  500.5 2022-07-03 17:29:02+00:00   
317      1201.31     0     0  None  None  500.5 2022-07-03 17:44:02+00:00   
318      1207.21     0     0  None  None  500.5 2022-07-03 18:00:25+00:00   
319      1214.94     0     0  None  None  500.5 2022-07-03 18:14:02+00:00   
320      1221.00     0     0  None  None  500.5 2022-07-03 18:30:13+00:00   

    station_id  
0         1156  
1         1156  
2         1156  
3      

In [22]:
s.result

,water_level,wspd,wdir,atmp,rh,pres,date_time,station_id
0,1040.94,0,0,28.9,76,1010.3,2022-04-27 21:41:55+00:00,876
1,1030.74,0,0,28.9,77,1010.4,2022-04-27 21:55:05+00:00,876
2,1017.98,0,0,28.8,77,1010.6,2022-04-27 22:10:06+00:00,876
3,1007.92,0,0,28.9,78,1010.7,2022-04-27 22:25:04+00:00,876
4,1001.21,0,0,28.0,81,1010.9,2022-04-27 22:41:16+00:00,876
...,...,...,...,...,...,...,...,...
334,1077.51,0,0,28.3,79,1012.1,2022-05-01 23:25:01+00:00,876
335,1061.74,0,0,28.5,79,1012.3,2022-05-01 23:40:01+00:00,876
336,1035.95,0,0,29.2,77,1012.6,2022-05-02 00:10:00+00:00,876
337,1023.66,0,0,29.1,78,1012.6,2022-05-02 00:26:11+00:00,876


In [43]:
s.result.drop(columns=columns, inplace=True)

In [44]:
s.result.columns = ['pres', 'atmp', 'rh', 'dewpt', 'wspd', 'wdir', 'gust', 'swvht', 'mxwvht', 'tp', 'sst', 'wvspread', 'wvdir', 'date_time']

In [45]:
s.result = s.result.replace(to_replace =['None', 'NULL', ' ', ''],
                                        value =np.nan)

In [46]:
s.result['station_id'] = 898

In [ ]:
s.